# Rollup of Total Grant Value for all US Counties

Grant information by state was compiled by [Investigate West](http://www.invw.org/2012/06/11/lwcf-grants-database-1283/) for the period 1965-2011.

In [2]:
import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [11]:
# Collect all file names and process into DataFrames
state_files = glob.glob('./Data/StateGrantData/*.xlsx')
dfs = []

for file in state_files:
    dfs.append(pd.read_excel(file))

all_states = pd.concat(dfs, ignore_index=True)
all_states.head()

,Grant ID & Element,Grant Name,Sponsor,County,State,Grant Amount,Year Approved,Year Completed,Type
0,3 - XXX,INDIAN PEAKS - BAKER CANYON,STATE OF UTAH,BEAVER,UT,5985.92,1966,1966,Development
1,4 - XXX,WEST BEAR RIVER BAY,STATE OF UTAH,BOX ELDER,UT,18500.00,1966,1967,Development
2,6 - XXX,WELLSVILLE PARK,CITY OF WELLSVILLE,CACHE,UT,934.25,1966,1967,Acquisition
3,2 - XXX,FARMINGTON BAY WATERFOWL AREA,STATE OF UTAH,DAVIS,UT,15150.00,1966,1967,Acquisition
4,11 - XXX,4 CITY PARKS,CITY OF BOUNTIFUL,DAVIS,UT,16225.15,1966,1966,Development


In [12]:
all_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41145 entries, 0 to 41144
Data columns (total 9 columns):
Grant ID & Element    41145 non-null object
Grant Name            41145 non-null object
Sponsor               41145 non-null object
County                41145 non-null object
State                 41145 non-null object
Grant Amount          41145 non-null float64
Year Approved         41145 non-null int64
Year Completed        41145 non-null int64
Type                  41145 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 2.8+ MB


In [16]:
all_states['State'].sort_values().unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [17]:
all_states['State'].nunique()

51

In [24]:
# Create grouped grant data
all_counties = all_states.groupby(['State', 'County'])['Grant Amount'].agg(['count', 'sum'])

# Reset multi-index to flatten columns
all_counties = all_counties.reset_index()

# Re-name columns
all_counties.columns = ['state', 'county_orig', 'num_grants', 'grant_value']

all_counties.head(5)

,state,county_orig,num_grants,grant_value
0,AK,ALEUTIANS EAST,5,123552.58
1,AK,ALEUTIANS WEST,1,52405.00
2,AK,ANCHORAGE,67,9475506.86
3,AK,BETHEL,3,366133.32
4,AK,DILLINGHAM,2,96119.80
5,AK,FAIRBANKS NORTH STAR,23,2765458.52
6,AK,HAINES,10,2978361.65
7,AK,JUNEAU,18,1945647.75
8,AK,KENAI PENINSULA,34,3032183.91
9,AK,KENAI-COOK INLET,1,20000.00


In [26]:
# Drop multi-county, planning, and no county rows
all_counties = all_counties[~all_counties['county_orig'].isin(['MULTI-COUNTY', 'PLANNING', 'NO COUNTY'])]

all_counties.head(20)

,state,county_orig,num_grants,grant_value
0,AK,ALEUTIANS EAST,5,123552.58
1,AK,ALEUTIANS WEST,1,52405.00
2,AK,ANCHORAGE,67,9475506.86
3,AK,BETHEL,3,366133.32
4,AK,DILLINGHAM,2,96119.80
5,AK,FAIRBANKS NORTH STAR,23,2765458.52
6,AK,HAINES,10,2978361.65
7,AK,JUNEAU,18,1945647.75
8,AK,KENAI PENINSULA,34,3032183.91
9,AK,KENAI-COOK INLET,1,20000.00


In [32]:
all_counties['county_orig'].count()

2989

In [30]:
# Import county id data
county_ids = pd.read_csv('./Data/county_names.csv', sep=',')
county_ids.head()

,id,state,county
0,1001,Alabama,Autauga County
1,1003,Alabama,Baldwin County
2,1005,Alabama,Barbour County
3,1007,Alabama,Bibb County
4,1009,Alabama,Blount County


In [31]:
county_ids['county'].count()

3219